<a href="https://colab.research.google.com/github/natalia-kurbakova/Baruch-Data-Challenge/blob/main/real_time_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Story


In [ ]:
import boto3
import urllib.request
import json
import datetime

s3 = boto3.client(service_name = 's3')

def lambda_handler(event, context):
    s3Bucket = 'citibikejsonfiles'

    url = "https://gbfs.citibikenyc.com/gbfs/en/station_status.json"
    url2 = "https://gbfs.citibikenyc.com/gbfs/en/station_information.json"
    try:
        responseStationStatus = urllib.request.urlopen(url)
        responseStationInformation = urllib.request.urlopen(url2)
    except Exception as e:
        print(f'Error while fetching data: {e}')
        return e
    try:
        dataStationStatus = json.loads(responseStationStatus.read().decode('utf-8'))
        dataStationInformation = json.loads(responseStationInformation.read().decode('utf-8'))
    except Exception as e:
        print(f'Error while parsing data: {e}')
        return e

    jsonFiles = {"dataStationStatus" : dataStationStatus, "dataStationInformation" : dataStationInformation}

    for i in jsonFiles:
        filename = i + " " + str(datetime.datetime.utcnow()) + ".json"
        json_data = json.dumps(jsonFiles[i])
        try:
            s3.put_object(Bucket=s3Bucket, Key=filename, Body=json_data)
        except Exception as e:
            print(f'Error while uploading data: {e}')
            return e
            
    print("Success! " + str(datetime.datetime.utcnow()))